In [46]:
import numpy as np
import random
from tqdm.auto import tqdm

# Multiple 1-0 knpasack problem
## Algorithm Overview

### 0. Initialize the first solution
Filling knapsacks with the most valuable items without exceeding weight limits

- Initialize empty knapsacks and a set of used items.
- For each knapsack:
  - Track remaining capacity.
  - Place items in descending order of value if they fit and haven't been used.
- Return the filled knapsack solution matrix.

### 1. Generate a neighbor solution
A neighbor solution is generated using one of the following tweaks:

1. **Invert an item** – change the presence of one item in a knapsack (0 → 1 or 1 → 0).  
2. **Move an item** – move an item from one knapsack to another.  
3. **Swap items** – exchange two items between different knapsacks.  

### 2. Evaluate the New Solution
- Compute `new_cost` (total value of items in the solution).  
- Compare `new_cost` with `current_cost`.
- Accept new solution if  `new_cost` >= `current_cost`

### 3. Simulated Annealing Decision
- Compute the difference:  
$$
\Delta = \text{new_cost} - \text{current_cost}
$$

- Compute temperature:
$$
T = \max(0.1, T_0 - \text{cooling_rate} \times \text{step})
$$

- Draw a random number `r` from `[0, 1]`.  
- **Accept new solution** if:
$$
r < P
$$


This allows accepting worse solutions to escape local optima.

### 4. Fallback Heuristic
With probability
$$
random(0, 1) < 0.5
$$
Check, if the new solution is rejected **and** no remaining items can be added to any knapsack:
- Remove the **least valuable item** from the current solution to free space.  
- Continue the algorithm with the updated solution.


In [54]:
def generate_maximally_filled_solution(values, weights, constraints):
    num_items = len(values)
    num_knapsacks = constraints.shape[0]

    solution = np.zeros((num_knapsacks, num_items), dtype=int)
    used_items = set()

    # Iterate over knapsacks one by one
    for k in range(num_knapsacks):
        remaining_capacity = constraints[k].copy()

        # Iterate over items in descending order of value
        for i in np.argsort(-values):
            if i in used_items:
                continue  # item is already placed in another knapsack

            item_weight = weights[i]
            if np.all(item_weight <= remaining_capacity):
                solution[k, i] = 1
                remaining_capacity -= item_weight
                used_items.add(i)

    return solution


In [45]:
# the constraints
def is_valid(solution, weights, constraints):
    if not np.all(solution.sum(axis=0) <= 1):
        return False

    for k in range(solution.shape[0]):
        selected = solution[k] == 1
        total_weight = np.sum(weights[selected], axis=0)
        if not np.all(total_weight <= constraints[k]):
            return False

    return True


In [47]:
# cost function that I try to maximize
def cost(solution, values):
    return np.dot(solution, values)

In [48]:
# neigbours generation
def tweak(solution, weights, constraints):
    new_solution = solution.copy()

    # flip a random item
    k = np.random.randint(NUM_KNAPSACKS)
    i = np.random.randint(NUM_ITEMS)
    new_solution[k, i] = 1 - new_solution[k, i]

    if is_valid(new_solution, weights, constraints):
        return new_solution

    # if flip didn't work, move on
    # move an item between knapsacks
    new_solution = solution.copy()
    current_k = np.where(solution[:, i] == 1)[0]
    if len(current_k) == 1:
        current_k = current_k[0]
        target_k = np.random.choice([kk for kk in range(NUM_KNAPSACKS) if kk != current_k])
        new_solution[current_k, i] = 0
        new_solution[target_k, i] = 1

        if is_valid(new_solution, weights, constraints):
            return new_solution

    # if both of previous methods didn't work, move on
    # swap two items between knapsacks
    new_solution = solution.copy()
    k1, k2 = np.random.choice(NUM_KNAPSACKS, size=2, replace=False)
    items_k1 = np.where(solution[k1] == 1)[0]
    items_k2 = np.where(solution[k2] == 1)[0]

    if len(items_k1) > 0 and len(items_k2) > 0:
        i1 = np.random.choice(items_k1)
        i2 = np.random.choice(items_k2)

        new_solution[k1, i1] = 0
        new_solution[k2, i2] = 0
        new_solution[k1, i2] = 1
        new_solution[k2, i1] = 1

        if is_valid(new_solution, weights, constraints):
            return new_solution

    # if none of the methods create a valid solution
    return solution

In [49]:
def no_addable_items_left(solution, weights, constraints):

    for i in range(NUM_ITEMS):
        if solution[:, i].any():  # item already in some knapsack
            continue

        for k in range(NUM_KNAPSACKS):
            used = solution[k].astype(bool)
            total_weight = weights[used].sum(axis=0) + weights[i]

            # if the item fits, we can still add something => return False
            if np.all(total_weight <= constraints[k]):
                return False

    # If we didn’t find any item that fits anywhere
    return True

def purge_least_valuable_item(solution, weights, constraints, values):
    # Compute value of each item currently in a knapsack
    item_values = np.where(solution.any(axis=0), values, np.inf)

    # Find the index of the least valuable item
    i_min = np.argmin(item_values)
    if not np.isfinite(item_values[i_min]):
        return solution  # no items to remove

    # Find which knapsack contains it and remove it
    k_min = np.argmax(solution[:, i_min])  # index of the knapsack
    solution[k_min, i_min] = 0

    return solution

In [66]:
# main function
def optimise(values, weights, constraints, max_steps, init_temp, cooling_rate):
    current_solution = generate_maximally_filled_solution(values, weights, constraints)
    current_cost = cost(current_solution, values)

    best_solution = current_solution.copy()
    best_cost = current_cost.copy()

    for step in tqdm(range(max_steps)):
        new_solution = tweak(current_solution, weights, constraints)
        new_cost = cost(new_solution, values)

        delta = new_cost.sum() - current_cost.sum()
        temperature = max(0.1, init_temp - cooling_rate * step)
        prob = np.exp(delta / temperature)

        # if found better solution
        if delta >= 0:
            current_solution, current_cost = new_solution.copy(), new_cost.copy()
            if current_cost.sum() > best_cost.sum():
                best_solution, best_cost = current_solution.copy(), current_cost.copy()
        elif np.random.rand() < prob: # accepted worse
            current_solution, current_cost = new_solution.copy(), new_cost.copy()
        elif np.random.rand() < 0.5: # with some probability
            if no_addable_items_left(current_solution, weights, constraints): # we are checking if there is no space for other items in all of knapsacks
                current_solution = purge_least_valuable_item(current_solution, weights, constraints, values)
                current_cost = cost(current_solution, values)

    print(f"Best total value: {best_cost.sum():.2f}")
    print("Best solution:\n", best_solution)


## TEST PROBLEMS

In [68]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [70]:
MAX_STEPS = 1000
TEMP_INIT = 50.0
COOLING_RATE = 0.0001

optimise(VALUES, WEIGHTS, CONSTRAINTS, MAX_STEPS, TEMP_INIT, COOLING_RATE)

  0%|          | 0/1000 [00:00<?, ?it/s]

Best total value: 1065.00
Best solution:
 [[0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0]
 [1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1]]


In [71]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [72]:
MAX_STEPS = 30000
TEMP_INIT = 50.0
COOLING_RATE = 0.0001

optimise(VALUES, WEIGHTS, CONSTRAINTS, MAX_STEPS, TEMP_INIT, COOLING_RATE)


  0%|          | 0/30000 [00:00<?, ?it/s]

Best total value: 50495.00
Best solution:
 [[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0

In [77]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [78]:
MAX_STEPS = 50000
initial_temp = 50.0
cooling_rate = 0.95

optimise(VALUES, WEIGHTS, CONSTRAINTS, MAX_STEPS, TEMP_INIT, COOLING_RATE)

  0%|          | 0/50000 [00:00<?, ?it/s]

Best total value: 1982175.00
Best solution:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
